## DiD Analysis

To study the effect of the policy change of minimum wages on employment rate using the DiD method, we can use the following extended DiD model:

$Y_{ist} = \alpha + \sum_{t \in T} \beta_t \times Post_t \times Treat_{st} + \gamma \times Post_t + \delta \times Treat_{st} + X'_{ist} \beta + \epsilon_{ist}$

where

$Y_{ist}$ is the outcome variable (employment rate 'emp_rate'),

$Post_t$ is a time dummy variable, which takes the value of 1 for the period after the policy change (for example, after 1997 if the policy change happened in 1997), and 0 for the period before the policy change,

$Treat_t$ is a dummy variable for whether the observation (state) is in the treatment group (those affected by the policy change),

$X'_{ist}$ are other control variables, which could include economic characteristics that affect employment, such as per capita GDP, unemployment rate, etc,

$Treat_{st}$ is assumed to remain unchanged throughout the period after the policy change.


### Prompt:

- To analyze the impact of minimum wage adjustments on the employment rate across different states and periods from 1993 to 2003 using an extended multi-period Difference-in-Differences (DiD) model, follow these steps in Python:

- Load your dataset containing the minimum wage (MW) and employment rate (emp_rate) for different states (statenum) over the years 1993-2003.

- Create a time dummy variable for each year after the policy change. If there's no specific year of policy change, consider creating a time dummy for each year in the dataset.

- Identify the states with minimum wage changes in each year and encode this information by creating a separate policy indicator variable for each state and year.

- Construct interaction terms between the time dummy variables and the policy indicator variables to capture the effect of the policy changes in different states and at different times.

- Include state fixed effects to control for time-invariant characteristics of each state and year fixed effects to control for common trends affecting all states.
- Estimate the model using ordinary least squares (OLS) regression, with the employment rate as the dependent variable and the interaction terms, state fixed effects, and year fixed effects as independent variables.
- Interpret the coefficients of the interaction terms to understand the impact of minimum wage changes on employment rates. A significant positive coefficient suggests that minimum wage increases are associated with an increase in the employment rate, while a significant negative coefficient suggests a decrease in employment rate due to minimum wage increases.

In [3]:

import pandas as pd
import statsmodels.formula.api as smf

# Load data
df = pd.read_csv(r"D:\desktop\24spring\ECMT680\midterm2\data\data_93-03.csv")

# Identify which states had at least one change in the minimum wage during the observation period
df['MW_change'] = df.groupby('statenum')['MW'].diff().fillna(0).astype(bool).astype(int)
df['Treat'] = df.groupby('statenum')['MW_change'].transform('max')

# Clean up unwanted middle columns
df.drop(columns='MW_change', inplace=True)

# Create an annual post-processing period indicator variable and interact with Treat
for year in range(1993, 2004):
    df[f'Post_{year}'] = (df['year'] >= year).astype(int)
    df[f'Post_{year}_Treat'] = df[f'Post_{year}'] * df['Treat']

# The formula of regression model is established
formula_parts = [f'Post_{year}_Treat' for year in range(1993, 2004)]
formula = f'emp_rate ~ {" + ".join(formula_parts)} + C(statenum) + C(year)'

# Fitting model
result = smf.ols(formula, data=df).fit()

# Print result
print(result.summary())




                            OLS Regression Results                            
Dep. Variable:               emp_rate   R-squared:                       0.812
Model:                            OLS   Adj. R-squared:                  0.812
Method:                 Least Squares   F-statistic:                 1.794e+04
Date:                Mon, 22 Apr 2024   Prob (F-statistic):               0.00
Time:                        23:13:28   Log-Likelihood:             5.4050e+05
No. Observations:              262548   AIC:                        -1.081e+06
Df Residuals:                  262484   BIC:                        -1.080e+06
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          3.922e+08   4.77e+0